In [1]:
import pandas as pd
import numpy as np

reviews_data = pd.read_csv("../data/raw/Books_rating.csv")

In [2]:
reviews_data.sample(5)

,Id,Title,Price,User_id,profileName,review/helpfulness,review/score,review/time,review/summary,review/text
2467783,B000PC3Q3G,East of Eden,NaN,A3QZJYDGQX2STX,"Stellar_Girl ""Stellar_Girl""",0/1,5.0,1127865600,Impressive,"This book was very, very good from beginning t..."
1731031,B000CC49MQ,Hunger Point : A Novel,6.0,A1R01S04H6C16R,Pink Books,3/4,4.0,1125014400,Good Book,"I really enjoyed this book, You feel and care ..."
2815865,B000PSS6FS,Being and Nothingness,NaN,NaN,NaN,42/43,5.0,930700800,Review from a layman,If you are just getting your feet wet in ontol...
1417942,B000PC5GVQ,Flowers in the Attic,NaN,NaN,NaN,1/1,5.0,908928000,what a great book!!!!,My opinoin of this book is that it is a very g...
1699016,B000J2IYSO,Hard Times,NaN,A363T30JZA3TRB,C-flo,0/0,3.0,1335916800,Hard Times,I needed this book for a college class. It was...


In [3]:
titles = reviews_data[["Id", "Title"]].drop_duplicates()
titles

,Id,Title
0,1882931173,Its Only Art If Its Well Hung!
1,0826414346,Dr. Seuss: American Icon
10,0829814000,Wonderful Worship in Smaller Churches
14,0595344550,Whispers of the Wicked Saints
46,0253338352,"Nation Dance: Religion, Identity and Cultural ..."
...,...,...
2999979,B00085PL4C,"Old Quebec,: The fortress of New France,"
2999980,0761989889,On the Life of Christ: Chanted Sermons by the ...
2999981,B000NSGW7E,Ghost Story
2999988,0255364520,An End to Welfare Rights: The Rediscovery of I...


In [4]:
useful_columns = [
    "Id",
    "User_id",
    "review/score",
    "review/time",
]
reviews_data = reviews_data[useful_columns]
reviews_data.sample(5)

,Id,User_id,review/score,review/time
2778180,B000PMAOFE,AZEAOJPSA0T8X,5.0,1295049600
617561,0938420844,A2K7WXT2M64M0J,5.0,1043366400
1293506,0590103903,A9MLW5RK39IMI,5.0,928540800
211733,0385469306,NaN,5.0,832636800
189263,0460112872,NaN,4.0,934243200


In [5]:
sample_user = "CUSTOM_USER"

In [6]:
fives = pd.DataFrame({
    "Id": [
        "B00005VSEM", "9562910334", "B000NLM1TO", "0394556550", "B000KBO2R6", "1421501244", "B000NDSX6C",
        "1400047951", "0262012111", "0722534159", "0521620368", "0262100851", "0521023203", "B00007FYCH",
        "0743272773", "B000GROP62", "B000HA3E06", "1557833486"
    ],
    "User_id": sample_user,
    "review/score": 5.0,
    "review/time": 1,
})
threes = pd.DataFrame({
    "Id": [
        "0694520187", "B0000CO4JZ", "0201616165", "B0006C2CAS", "B00087AQAO", "9686769005", "B000HVWGZ4",
        "184046254X", "B000EHHOVC", "1892051001"
    ],
    "User_id": sample_user,
    "review/score": 3.0,
    "review/time": 1,
})
zeros = pd.DataFrame({
    "Id": [
        "B000N5HFQY", "B000PHN85C", "B000GSDG8E", "B000BARVK6", "B000H58GKO", "B000L3V6AO", "B0006CNZ0O",
        "034546236X", "B000ETWJ74", "0740716905", "B000PH24VQ", "0449234088", "B00005X5VG", "9999275109",
    ],
    "User_id": sample_user,
    "review/score": 0.0,
    "review/time": 1,
})

In [7]:
titles[titles.Title.str.contains("the last question", case=False).fillna(False)]

,Id,Title


In [8]:
reviews_data = pd.concat([reviews_data, fives, threes, zeros])

In [9]:
reviews_data = reviews_data[reviews_data.User_id.notna()]

In [10]:
M = reviews_data.Id.unique().shape[0]
print(f"The number of items M = {M}")

The number of items M = 216023


In [11]:
N = reviews_data.User_id.unique().shape[0]
print(f"The number of users N = {N}")

The number of users N = 1008973


In [12]:
print(f"The size of the sparse weight matrix is MxN, which contains {M * N} cells")

The size of the sparse weight matrix is MxN, which contains 217961374379 cells


In [13]:
def calculate_array_size(data_type, num_items):
    bytes_per_item = np.dtype(data_type).itemsize
    total_bytes = bytes_per_item * num_items
    total_gb = total_bytes / (1024**3)
    return total_gb

# Usage example
data_type = np.int8
num_items = M*N
array_size_in_gb = calculate_array_size(data_type, num_items)

print(f"The array will be approximately {array_size_in_gb:.2f} GB")

The array will be approximately 202.99 GB


We have a lot of items and users. We will need an efficient way of performing the calculations

Possible Filters for a pair of i and i' users:
- Threshold of common ratings: If two users have less than a certain amount of common reviewed items, they won't be considered.
- Threshold of weights: If the absolute value of the weight for two users is less than a threshold, it won't be considered.
- Theshold of neighbors: Only the top absolute value weights for a user will be considered.

We still don't know the distributions of many of these variables, but we can get an upper bound using the threshold of neighbors

In [14]:
pd.DataFrame(
    [(n,calculate_array_size(data_type, N * n)) for n in range(1,520,20)],
    columns=["neighbors", "size (GB)"]
)

,neighbors,size (GB)
0,1,0.000940
1,21,0.019733
2,41,0.038527
3,61,0.057320
4,81,0.076114
5,101,0.094908
6,121,0.113701
7,141,0.132495
8,161,0.151288
9,181,0.170082


We can see that the use of this threshold already gives us a size that we can work on

## Eliminating multiple reviews

In [15]:
reviews_data.groupby(["User_id", "Id"]).size().max()

28

We can see that a user can have many reviews per book. We only consider the last review.

In [16]:
reviews_data = reviews_data.sort_values('review/time', ascending=False)
reviews_data = reviews_data.drop_duplicates(subset=['Id', 'User_id'], keep='first')

In [17]:
reviews_data.groupby(["User_id", "Id"]).size().max()

1

## Removing users with repeating scores

In [18]:
reviews_data

,Id,User_id,review/score,review/time
2806366,1840323027,A35O6DX8L99DW0,5.0,1362355200
960829,1850891648,A120XIKYSR1RWG,4.0,1362355200
2572565,0140860428,A2IIFK4MOI2XWB,5.0,1362355200
2572566,0140860428,AI437UN6Z43GK,5.0,1362355200
1531279,0736693408,A3DWFPH8Q9XKIR,3.0,1362355200
...,...,...,...,...
2971444,B000G167FA,A3OK5K6RFO19JB,5.0,-1
2971443,B000G167FA,AWVWX5F3YEJKZ,5.0,-1
2152018,B000KPX7RI,A1AY4QM3FDINBQ,5.0,-1
75745,0786280670,A3LL5TMGX00LA1,2.0,-1


In [19]:
distinct_score_per_user = reviews_data.groupby("User_id")["review/score"].nunique()
distinct_score_per_user

User_id
A00109803PZJ91RLT7DPN    1
A00117421L76WVWG4UX95    1
A0015610VMNR0JC9XVL1     2
A002258237PFYJV336T05    1
A00264602WCXBHHFPLTQ4    1
                        ..
AZZZJY3RMN57G            1
AZZZT14MS21I6            1
AZZZYCR4NZADZ            1
AZZZZW74AAX75            1
CUSTOM_USER              3
Name: review/score, Length: 1008973, dtype: int64

In [20]:
reviews_data = reviews_data[reviews_data.User_id.isin(distinct_score_per_user[distinct_score_per_user>1].index)]

## Standardizing reviews by user

In [21]:
mean_score = reviews_data.groupby("User_id")["review/score"].mean()

In [22]:
reviews_data = pd.merge(
    reviews_data,
    mean_score.reset_index(name="user_average"),
    how="inner"
)

In [23]:
reviews_data["dev"] = reviews_data["review/score"] - reviews_data.user_average

## Working with a sample

In this section we will work with a sample of users with the highest count of reviews, to minimize the sparcity of the weight matrix.

In [24]:
top_active_users = reviews_data.User_id.value_counts().head(int(N/100)).index

In [25]:
top_active_users

Index(['A14OJS0VWMOSWO', 'AFVQZQ8PW0L', 'AHD101501WCN1', 'A1X8VZWTOG8IS6',
       'A1K1JW1C5CUSUZ', 'A1S3C5OFU508P3', 'A1N1YEMTI9DJ86', 'A2OJW07GQRNJUT',
       'A1D2C0WDCSHUWZ', 'A1G37DFO8MQW0M',
       ...
       'A28QXTQP2PSJB9', 'A3RIZM28X4VDE7', 'A38ZTF3PTQISH0', 'A1B4TEX216CAVA',
       'A3O6IY4UKCMQHG', 'A2YDHA8AA57ZYK', 'A575YWESG77HC', 'A3L4HTJCZII9XA',
       'A1EKGL0HGYJRB9', 'A2TWLP80OY28S6'],
      dtype='object', name='User_id', length=10089)

This helps us to find which users reviewed a book

In [26]:
reviewed_books = reviews_data[reviews_data.User_id == sample_user].Id

In [27]:
N_INTERSECTION_CANDIDATES = 10000

In [28]:
highest_intersection_users = (
    reviews_data[reviews_data.Id.isin(reviewed_books) & ~reviews_data.User_id.str.fullmatch(sample_user)]
    .User_id
    .value_counts()
    .nlargest(N_INTERSECTION_CANDIDATES)
)
highest_intersection_users

User_id
A1X8VZWTOG8IS6    8
A20EEWWSFMZ1PN    7
A1S3C5OFU508P3    7
A3DF7LMEQOND63    7
AHD101501WCN1     7
                 ..
A1F8LLJC3PQFZZ    1
AWY889718UHI5     1
AQL8Y0QLBR54N     1
A28LLREV8AUBGL    1
AYA9WPN56336D     1
Name: count, Length: 4148, dtype: int64

In [29]:
relevant_dev_matrix = reviews_data[
    reviews_data.User_id.isin(highest_intersection_users.index.tolist() + [sample_user]) &
    reviews_data.Id.isin(reviewed_books)
][
    ["User_id", "Id", "dev"]
].pivot(index="Id",columns="User_id", values="dev")

In [30]:
relevant_dev_matrix

User_id,A100YHBWL4TR4D,A103J1XV97K0TI,A1078PRFLW3YHN,A107DY7OT9IL6,A1085BVEWL9KAA,A10872FHIJAKKD,A1088GK8SE4M1Z,A10A5QVU9V778I,A10ARPP5YIQNKY,A10B4UOL0IB274,...,AZRFX7G4WX2RE,AZS4W8TUOI0D9,AZSOEKOUL0BNO,AZT60HLHFL0V0,AZTATSGF6ZLD3,AZVZSGHKV0AO0,AZW1Z3FJTV66W,AZX7FSZ98E9HV,AZY67T7FDKAOR,CUSTOM_USER
Id,,,,,,,,,,,,,,,,,,,,,
0201616165,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.142857
0262012111,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.142857
0262100851,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.142857
034546236X,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-2.857143
0394556550,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.142857
0449234088,NaN,NaN,-0.666667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-2.857143
0521023203,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.142857
0521620368,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.142857
0694520187,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.142857


In [31]:
relevant_dev_matrix.dropna(subset=[sample_user]).corrwith(relevant_dev_matrix[sample_user], method='pearson')

/Users/jorge.duenas/miniconda3/envs/book_recommender/lib/python3.8/site-packages/numpy/lib/function_base.py:2846: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
/Users/jorge.duenas/miniconda3/envs/book_recommender/lib/python3.8/site-packages/numpy/lib/function_base.py:2705: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)


User_id
A100YHBWL4TR4D    NaN
A103J1XV97K0TI    NaN
A1078PRFLW3YHN    NaN
A107DY7OT9IL6     NaN
A1085BVEWL9KAA    NaN
                 ... 
AZVZSGHKV0AO0     NaN
AZW1Z3FJTV66W     NaN
AZX7FSZ98E9HV     NaN
AZY67T7FDKAOR     NaN
CUSTOM_USER       1.0
Length: 4149, dtype: float64

In [32]:
weights = relevant_dev_matrix.corrwith(relevant_dev_matrix[sample_user])

/Users/jorge.duenas/miniconda3/envs/book_recommender/lib/python3.8/site-packages/numpy/lib/function_base.py:2846: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
/Users/jorge.duenas/miniconda3/envs/book_recommender/lib/python3.8/site-packages/numpy/lib/function_base.py:2705: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)


In [33]:
weights[weights.abs() > 0.8]

User_id
A10NF78Y1L3PE     1.000000
A10T0OW97SFBB     1.000000
A11YOT86X3M4GU    0.918559
A12G0C1T78B7S5    1.000000
A12GQKJI1ARMB7    1.000000
                    ...   
ATN1SSKTJD8Z8     1.000000
AUDSM2CTLLW1Q     1.000000
AWE0R9MSS9988    -1.000000
AXOA9OI962P0Q     1.000000
CUSTOM_USER       1.000000
Length: 168, dtype: float64

Problem with just using the correlation coefficients is that some correlations have greater weight than others. We will proceed to use the confidence intervals, specifically, the lower bound.

In [34]:
from scipy import stats

In [43]:
# Assuming df is your DataFrame and 'target_column' is the column you are interested in
correlations = {}
confidence_intervals = {}

for col in relevant_dev_matrix.columns:
    if col != sample_user:
        non_nulls = relevant_dev_matrix[[col, sample_user]].dropna()
        
        constant_col = non_nulls[col].unique().shape[0] == 1
        constant_target = non_nulls[sample_user].unique().shape[0] == 1
        if len(non_nulls) > 2 and not constant_col and not constant_target:  # Ensure there are at least 3 non-null pairs to calculate correlation
            corr, p_value = stats.pearsonr(non_nulls[col], non_nulls[sample_user])
            
            # Assuming you are calculating a 95% confidence interval
            abs_corr = abs(corr)

            z = 1.96  # Z-score for 95% confidence interval
            n = len(non_nulls)
            ci = z * ((abs_corr * (1 - abs_corr)) / (n - 1))**0.5  # Formula for confidence interval
            
            if abs_corr - ci > 0.1:
                correlations[col] = abs_corr
                confidence_intervals[col] = (abs_corr - ci, abs_corr + ci)

In [44]:
weight_df = pd.DataFrame(
    [(k,v[0]) for k,v in confidence_intervals.items()],
    columns=["User_id", "weight"]
)
weight_df

,User_id,weight
0,A10NF78Y1L3PE,1.000000
1,A11YOT86X3M4GU,0.650517
2,A132FOGSE0HDBL,1.000000
3,A134FLH2KNA0JU,1.000000
4,A13INPXFUIZNKH,1.000000
...,...,...
94,ARH9IQEMFKR6M,1.000000
95,ARU2H2PYTNCAX,1.000000
96,ARYEAKJSIATCP,1.000000
97,ATN1SSKTJD8Z8,1.000000


In [45]:
relevant_reviews = reviews_data[reviews_data.User_id.isin(weight_df.User_id)][["Id","User_id","dev"]].copy()
relevant_reviews.sample(5)

,Id,User_id,dev
565234,039457186X,ACX80IV6CGBZ0,0.629213
123007,0582527988,A1S3C5OFU508P3,-0.358704
641695,B000NPEWHE,A1LVMQ52YODRMO,0.568182
123387,081614284X,A1S3C5OFU508P3,-2.358704
101,0140860282,A2DKTZMMG3JHN4,0.106383


In [46]:
a = pd.merge(
    relevant_reviews,
    weight_df,
    how="inner",
)
a

,Id,User_id,dev,weight
0,068199570X,A2DKTZMMG3JHN4,0.106383,1.0
1,0606015825,A2DKTZMMG3JHN4,0.106383,1.0
2,1593351348,A2DKTZMMG3JHN4,0.106383,1.0
3,1582790337,A2DKTZMMG3JHN4,0.106383,1.0
4,1569602093,A2DKTZMMG3JHN4,0.106383,1.0
...,...,...,...,...
14957,B000NXH1RE,A3BDCUGKKKFTJO,0.250000,1.0
14958,9562910334,A3BDCUGKKKFTJO,0.250000,1.0
14959,B000HVR6KY,A3BDCUGKKKFTJO,0.250000,1.0
14960,B000NHNM3C,A3BDCUGKKKFTJO,0.250000,1.0


In [47]:
a["exp_val_lb"] = a.dev * a.weight
a

,Id,User_id,dev,weight,exp_val_lb
0,068199570X,A2DKTZMMG3JHN4,0.106383,1.0,0.106383
1,0606015825,A2DKTZMMG3JHN4,0.106383,1.0,0.106383
2,1593351348,A2DKTZMMG3JHN4,0.106383,1.0,0.106383
3,1582790337,A2DKTZMMG3JHN4,0.106383,1.0,0.106383
4,1569602093,A2DKTZMMG3JHN4,0.106383,1.0,0.106383
...,...,...,...,...,...
14957,B000NXH1RE,A3BDCUGKKKFTJO,0.250000,1.0,0.250000
14958,9562910334,A3BDCUGKKKFTJO,0.250000,1.0,0.250000
14959,B000HVR6KY,A3BDCUGKKKFTJO,0.250000,1.0,0.250000
14960,B000NHNM3C,A3BDCUGKKKFTJO,0.250000,1.0,0.250000


In [48]:
a = a.groupby("Id")[["weight","exp_val_lb"]].sum()

In [49]:
a["score"] = a.exp_val_lb / a.weight
a

,weight,exp_val_lb,score
Id,,,
0001050079,1.000000,0.855219,0.855219
0001050087,1.000000,0.855219,0.855219
0001055003,1.000000,0.515625,0.515625
0002155400,3.571775,1.810228,0.506815
0002222876,1.000000,0.855219,0.855219
...,...,...,...
B000U6J67M,1.000000,-0.282297,-0.282297
B000UDFI40,0.536702,0.235267,0.438356
B000UDFI6I,0.536373,0.162345,0.302672


In [50]:
top = a.sort_values("score", ascending=False).head(50)
pd.merge(
    titles,
    top,
    left_on="Id",
    right_index=True,
    how="inner",
)

,Id,Title,weight,exp_val_lb,score
44216,B000K9UV8C,Desert Solitaire: A Season in the Wilderness,1.000000,1.839286,1.839286
96379,1593375794,The Everything Texas Hold 'Em Book: Tips And T...,1.000000,1.380435,1.380435
114452,B000HWI3L4,Star Trek: Federation,1.000000,1.379679,1.379679
137544,B000K0C0IK,The Day Lincoln was Shot,1.000000,1.388060,1.388060
167883,B000PB01P8,Sharra's Exile: A Darkover Novel,1.000000,1.379679,1.379679
332850,B000JWKOAA,Goodbye to a River: a Narrative,1.000000,1.839286,1.839286
338781,B0007EHI2M,Cape Cod (Apollo editions),1.000000,1.839286,1.839286
404005,0788751530,The Girl Who Loved Tom Gordon,1.000000,1.380435,1.380435
493779,1411400836,Vampire Dreams (Smart Novels: Vocabulary),1.000000,1.380435,1.380435
527036,B0008CSR8M,Old Tippecanoe: William Henry Harrison and his...,1.000000,1.379679,1.379679
